In [2]:
import requests  
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import json
import re
import time
import datetime
import pprint
import ast

In [60]:
test = pd.read_csv('test.csv')
test.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [61]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [259]:
brands = [x.lower() for x in test.brand.value_counts().index] 
print(brands)

['bmw', 'volkswagen', 'nissan', 'mercedes', 'toyota', 'audi', 'mitsubishi', 'skoda', 'volvo', 'honda', 'infiniti', 'lexus']


In [164]:
brands = test.brand.unique()
cars_urls = {}
print(brands)

['SKODA' 'AUDI' 'HONDA' 'VOLVO' 'BMW' 'NISSAN' 'INFINITI' 'MERCEDES'
 'TOYOTA' 'LEXUS' 'VOLKSWAGEN' 'MITSUBISHI']


In [165]:
url = 'https://auto.ru/moskva/cars/'
for car_brand in brands:
    brand_url = url + car_brand + '/used/?page='
    cars_urls[car_brand] = []
    print(car_brand)
    for number in range(1, 100):
        response = requests.get(brand_url+str(number), headers={'User-Agent': 'Mozilla/5.0'})
        page = BeautifulSoup(response.text, 'html.parser')
        link_list = page.find_all('a', class_='Link ListingItemTitle-module__link')
        if len(link_list) !=0:
            for link in link_list:
                cars_urls[car_brand].append(link['href'])
        else:
            break
with open('data.json', 'w') as fp:
    json.dump(cars_urls, fp)

SKODA
AUDI
HONDA
VOLVO
BMW
NISSAN
INFINITI
MERCEDES
TOYOTA
LEXUS
VOLKSWAGEN
MITSUBISHI


In [262]:
with open('data.json') as fp:
    cars_urls = json.load(fp)

In [263]:
type(cars_urls)

dict

In [264]:
cars_urls.keys()

dict_keys(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI', 'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'])

In [265]:
print(len(cars_urls['SKODA']))

2354


In [268]:
cars_urls['SKODA'][2]

'name'

In [189]:
# пробовала на одной страничке 
''''url_1 = text['SKODA'][0]
response_1 = requests.get(url_1, headers={'User-Agent': 'Mozilla/5.0'})  
response_1.encoding = 'utf8'
soup = BeautifulSoup(response_1.text, 'html.parser')  
car_info = {}
cls_str = 'CardInfoRow_'
sd = 'sale-data-attributes'


car_info['bodyType'] = soup.find(
            'li', class_=cls_str+'bodytype').find('a').text.split(' ')[0]
car_info['color'] = soup.find(
            'li', class_=cls_str+'color').find('a').text
car_info['description'] = soup.find(
            'div', class_='CardDescription__textInner').text
car_info['engine'] = soup.find(
            'li', class_=cls_str+'engine').find('div').text
car_info['engineDisplacement'], car_info['enginePower'], car_info['fuelType'] = car_info['engine'].split('/')
car_info['equipment_dict'] = sale_data = json.loads(
            soup.find(id=sd)['data-bem'])[sd]
car_info['mileage'] = soup.find(
            'li', class_=cls_str+'kmAge').find_all('span')[1].text
soup_name = soup.find(
            'div', class_='CardBreadcrumbs').find_all('a')
car_info['card_type'] = soup_name[1].text
car_info['brand'] = soup_name[2].text
car_info['model_name'] = soup_name[3].text
car_info['model_name2'] = soup_name[4].text
car_info['name'] = soup_name[6].text

#car_info['numberOfDoors'] = soup.find('li', class_=cls_str+'bodytype').find('a').text.split(' ')[1]
car_info['datetime'] = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=3)))
car_info['parsing_unixtime'] = int(time.time())
car_info['productionDate'] = soup.find('li', class_=cls_str+'year').find_all('span')[1].text
car_info['sell_id'] = soup.find('div', title='Идентификатор объявления').text
car_info['vehicleTransmission'] = soup.find(
            'li', class_=cls_str+'transmission').find_all('span')[1].text
car_info['Владельцы'] = soup.find(
            'li', class_=cls_str+'ownersCount').find_all('span')[1].text
car_info['Владение'] = soup.find('li', class_=cls_str+'owningTime').find_all('span')[1].text
car_info['ПТС'] = soup.find(
            'li', class_=cls_str+'pts').find_all('span')[1].text
car_info['Привод'] = soup.find(
            'li', class_=cls_str+'drive').find_all('span')[1].text
car_info['Руль'] = soup.find(
            'li', class_=cls_str+'wheel').find_all('span')[1].text
car_info['Состояние'] = soup.find(
            'li', class_=cls_str+'state').find_all('span')[1].text
car_info['Таможня'] = soup.find(
            'li', class_=cls_str+'customs').find_all('span')[1].text
#car_info['card'] = soup.find('div', class_='CardSidebarActions__title').text
#car_info['region'] = soup.find('span', class_='MetroListPlace__regionName MetroListPlace_nbsp').text
car_info['price'] = soup.find(
            'span', class_='OfferPriceCaption__price').text

print(car_info)''

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-189-2a5687c321b1>, line 56)

## Теперь проходимся циклом 

In [223]:
final_dict={}
final_dict['car'] = []

In [267]:
counter = 0

final_dict['car'] = []

for url in cars_urls['SKODA'][:]:
    car_info = {}
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    response.encoding = 'utf8'
    soup = BeautifulSoup(response.text, 'html.parser')  
    
    cls_str = 'CardInfoRow_'
    sd = 'sale-data-attributes'
    
    if 'Этот автомобиль уже продан' in soup.text:
        continue
    else:
        try:
            cars_urls['SKODA'][:] = ast.literal_eval(
                str(soup.find_all('script', attrs={'type': 'application/ld+json'})[0])[35:-9])
        except IndexError:
            continue
        url2 = ast.literal_eval(str(soup.find_all(
            'script', attrs={'type': 'application/ld+json'})[0])[35:-9])['offers']['url']
        if url != url2:
            continue
        else:
            try:
                car_info['bodyType'] = soup.find('li', class_=cls_str+'bodytype').find('a').text
                car_info['color'] = soup.find('li', class_=cls_str+'color').find('a').text
                car_info['description'] = soup.find('div', class_='CardDescription__textInner').text
                car_info['engine'] = soup.find('li', class_=cls_str+'engine').find('div').text
                car_info['engineDisplacement'], car_info['enginePower'], car_info['fuelType'] = car_info['engine'].split('/')
                car_info['equipment_dict'] = sale_data = json.loads(soup.find(id=sd)['data-bem'])[sd]
                car_info['mileage'] = soup.find('li', class_=cls_str+'kmAge').find_all('span')[1].text
                soup_name = soup.find('div', class_='CardBreadcrumbs').find_all('a')
                car_info['card_type'] = soup_name[1].text
                car_info['brand'] = soup_name[2].text
                car_info['model_name'] = soup_name[3].text
                car_info['model_name2'] = soup_name[4].text
                car_info['name'] = soup_name[6].text
                #car_info['numberOfDoors'] = soup.find('li', class_=cls_str+'bodytype').find('a').text.split(' ')[1]
                car_info['datetime'] = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=3)))
                car_info['parsing_unixtime'] = int(time.time())
                car_info['productionDate'] = soup.find('li', class_=cls_str+'year').find_all('span')[1].text
                car_info['sell_id'] = soup.find('div', title='Идентификатор объявления').text
                car_info['vehicleTransmission'] = soup.find('li', class_=cls_str+'transmission').find_all('span')[1].text
                car_info['Владельцы'] = soup.find('li', class_=cls_str+'ownersCount').find_all('span')[1].text
                car_info['Владение'] = soup.find('li', class_=cls_str+'owningTime').find_all('span')[1].text
                car_info['ПТС'] = soup.find('li', class_=cls_str+'pts').find_all('span')[1].text
                car_info['Привод'] = soup.find('li', class_=cls_str+'drive').find_all('span')[1].text
                car_info['Руль'] = soup.find('li', class_=cls_str+'wheel').find_all('span')[1].text
                car_info['Состояние'] = soup.find('li', class_=cls_str+'state').find_all('span')[1].text
                car_info['Таможня'] = soup.find('li', class_=cls_str+'customs').find_all('span')[1].text
                car_info['price'] = soup.find('span', class_='OfferPriceCaption__price').text
                final_dict['car'].append(car_info)
                counter += 1
                if counter % 100 == 0:
                    print(counter)
            except (KeyError, ValueError, AttributeError) as e:
                continue
                
print('end')

100
200
300
400
500
600
end


In [269]:
final_dict

{'car': [{'bodyType': 'лифтбек',
   'color': 'жёлтый',
   'description': 'Остался один автомобиль!!! Автомобили были замечены в такси. После тщательного подбора из 60 автомобилей парка, оставили 5 автомобилей. В эксплуатации с ноября 2018!!! После объёмного комплекса работ приведены в отличное техническое состояние и приятный внешний вид. Один хозяин, птс оригинал, два ключа. С документами полный порядок! Любые проверки! Уверены, что автомобили будут радовать новых владельцев ещё не одну сотню тысяч км. ',
   'engine': '1.6 л / 110\xa0л.с. / Бензин',
   'engineDisplacement': '1.6 л ',
   'enginePower': ' 110\xa0л.с. ',
   'fuelType': ' Бензин',
   'equipment_dict': {'asciiCat': 'cars',
    'category': 'cars',
    'engine-type': 'GASOLINE',
    'image': 'https://autoru.naydex.net/muw1x2723/0a4b9cL8fRgV/oZ2frms_h6gcJl5ssUk5CwzMNHHI9ac2dC48rBHql3CS2uKHG5WoX4-hMSLu8sns2iN7MdnDDHCeSIJXPRkpdPQJewsfb_PIaHcPmUEoaP-MmCEkvHRGU7Bvzy-yvFVCMWkxND3WKL_qFID2-tRfTZ4Hj0WeA_h1ay1kMAyciQakWanOai10qPxHOb0

In [270]:
tp1 = pd.DataFrame()
for i in range(0, (len(final_dict['car']))):
    print(i)
    tp2 = pd.DataFrame(final_dict['car'][i].values(), index=final_dict['car'][0].keys()).T
    tp1 = tp1.append(tp2, ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [271]:
tp1

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,лифтбек,жёлтый,Остался один автомобиль!!! Автомобили были зам...,1.6 л / 110 л.с. / Бензин,1.6 л,110 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",125 000 км,Б/у,...,№ 1103050112,автоматическая,1 владелец,2 года и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,930 000 ₽
1,компактвэн,серый,"Хороший рабочий автомобиль, удобный для семьи....",1.6 л / 105 л.с. / Бензин,1.6 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",158 000 км,Б/у,...,№ 1103351952,механическая,2 владельца,8 лет и 3 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,490 000 ₽
2,внедорожник 5 дв.,серый,Здравствуйте! Предлагаю вашему вниманию достой...,1.8 л / 152 л.с. / Бензин,1.8 л,152 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",44 500 км,Б/у,...,№ 1102830688,роботизированная,2 владельца,2 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 350 000 ₽
3,лифтбек,синий,"Продам автомобиль свой личный,куплен в сентябр...",1.6 л / 110 л.с. / Бензин,1.6 л,110 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",40 000 км,Б/у,...,№ 1103503095,автоматическая,1 владелец,1 год и 8 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 080 000 ₽
4,лифтбек,белый,Все исправно работает! Вся в родной краске. Ку...,"1.6 л / 110 л.с. / Бензин, газобаллонное обору...",1.6 л,110 л.с.,"Бензин, газобаллонное оборудование","{'asciiCat': 'cars', 'category': 'cars', 'engi...",150 000 км,Б/у,...,№ 1103503093,автоматическая,2 владельца,2 года,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 035 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,лифтбек,фиолетовый,"Не думал продавать, но подвернулась замена.Оче...",1.6 л / 101 л.с. / Бензин,1.6 л,101 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",425 000 км,Б/у,...,№ 1103305091,механическая,3 или более,4 года и 7 месяцев,Дубликат,передний,Левый,Не требует ремонта,Растаможен,180 000 ₽
692,лифтбек,серебристый,Продам личный Авто Октавия А5. Состояние очень...,1.8 л / 152 л.с. / Бензин,1.8 л,152 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",147 500 км,Б/у,...,№ 1103245625,роботизированная,3 или более,1 год,Дубликат,передний,Левый,Не требует ремонта,Растаможен,490 000 ₽
693,хэтчбек 5 дв.,бежевый,Автомобиль в отличном состоянии для своих лет....,1.2 л / 70 л.с. / Бензин,1.2 л,70 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",180 200 км,Б/у,...,№ 1103366330,механическая,1 владелец,7 лет и 6 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,469 000 ₽
694,лифтбек,пурпурный,"Автомобиль чешской сборки, оцинкованный. В хор...",1.6 л / 102 л.с. / Бензин,1.6 л,102 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",229 000 км,Б/у,...,№ 1103362473,механическая,3 или более,1 год и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,350 000 ₽


In [234]:
train = tp1.to_csv('project6_train_5.csv', index = False)

In [236]:
train = pd.read_csv('project6_train_5.csv')
train

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,лифтбек,жёлтый,Остался один автомобиль!!! Автомобили были зам...,1.6 л / 110 л.с. / Бензин,1.6 л,110 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",125 000 км,Б/у,...,№ 1103050112,автоматическая,1 владелец,2 года и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,930 000 ₽
1,компактвэн,серый,"Хороший рабочий автомобиль, удобный для семьи....",1.6 л / 105 л.с. / Бензин,1.6 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",158 000 км,Б/у,...,№ 1103351952,механическая,2 владельца,8 лет и 3 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,490 000 ₽
2,внедорожник 5 дв.,серый,Здравствуйте! Предлагаю вашему вниманию достой...,1.8 л / 152 л.с. / Бензин,1.8 л,152 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",44 500 км,Б/у,...,№ 1102830688,роботизированная,2 владельца,2 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 350 000 ₽
3,лифтбек,синий,"Продам автомобиль свой личный,куплен в сентябр...",1.6 л / 110 л.с. / Бензин,1.6 л,110 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",40 000 км,Б/у,...,№ 1103503095,автоматическая,1 владелец,1 год и 8 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 080 000 ₽
4,лифтбек,белый,Все исправно работает! Вся в родной краске. Ку...,"1.6 л / 110 л.с. / Бензин, газобаллонное обору...",1.6 л,110 л.с.,"Бензин, газобаллонное оборудование","{'asciiCat': 'cars', 'category': 'cars', 'engi...",150 000 км,Б/у,...,№ 1103503093,автоматическая,2 владельца,2 года,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 035 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,лифтбек,фиолетовый,"Не думал продавать, но подвернулась замена.Оче...",1.6 л / 101 л.с. / Бензин,1.6 л,101 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",425 000 км,Б/у,...,№ 1103305091,механическая,3 или более,4 года и 7 месяцев,Дубликат,передний,Левый,Не требует ремонта,Растаможен,180 000 ₽
696,лифтбек,серебристый,Продам личный Авто Октавия А5. Состояние очень...,1.8 л / 152 л.с. / Бензин,1.8 л,152 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",147 500 км,Б/у,...,№ 1103245625,роботизированная,3 или более,1 год,Дубликат,передний,Левый,Не требует ремонта,Растаможен,490 000 ₽
697,хэтчбек 5 дв.,бежевый,Автомобиль в отличном состоянии для своих лет....,1.2 л / 70 л.с. / Бензин,1.2 л,70 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",180 200 км,Б/у,...,№ 1103366330,механическая,1 владелец,7 лет и 6 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,469 000 ₽
698,лифтбек,пурпурный,"Автомобиль чешской сборки, оцинкованный. В хор...",1.6 л / 102 л.с. / Бензин,1.6 л,102 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",229 000 км,Б/у,...,№ 1103362473,механическая,3 или более,1 год и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,350 000 ₽


In [244]:
train.brand.unique()

array(['Skoda'], dtype=object)

In [3]:
df_scoda = pd.read_csv('scoda_1.csv')

In [47]:
df_scoda

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,лифтбек,жёлтый,Остался один автомобиль!!! Автомобили были зам...,1.6 л / 110 л.с. / Бензин,1.6 л,110 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",125 000 км,Б/у,...,№ 1103050112,автоматическая,1 владелец,2 года и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,930 000 ₽
1,лифтбек,серый,"Комплектация ""HOCKEY EDITION"". Куплен в конце ...",1.4 л / 150 л.с. / Бензин,1.4 л,150 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",22 143 км,Б/у,...,№ 1102189226,роботизированная,1 владелец,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,1 545 000 ₽
2,компактвэн,серый,"Хороший рабочий автомобиль, удобный для семьи....",1.6 л / 105 л.с. / Бензин,1.6 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",158 000 км,Б/у,...,№ 1103351952,механическая,2 владельца,8 лет и 3 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,490 000 ₽
3,универсал 5 дв.,серебристый,Автомобиль в отличном состоянии! Обслуживался ...,1.8 л / 150 л.с. / Бензин,1.8 л,150 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",161 000 км,Б/у,...,№ 1102867943,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,265 000 ₽
4,лифтбек,чёрный,Наш автосалон обеспечивает максимальную безопа...,1.6 л / 115 л.с. / Бензин,1.6 л,115 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",152 654 км,Б/у,...,№ 1102818901,автоматическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,335 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,хэтчбек 5 дв.,синий,Продаю автомобиль шкода фабияМашина в хорошем ...,1.4 л / 86 л.с. / Бензин,1.4 л,86 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",134 648 км,Б/у,...,№ 1103373486,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,360 000 ₽
2284,хэтчбек 5 дв.,пурпурный,"Продается классный, маневренный, удобный автом...",1.4 л / 86 л.с. / Бензин,1.4 л,86 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",119 000 км,Б/у,...,№ 1103125723,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,425 000 ₽
2285,лифтбек,серый,"Skoda Octavia Tour 2009 года 1,8Т МКПП передел...",1.8 л / 150 л.с. / Бензин,1.8 л,150 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",215 500 км,Б/у,...,№ 1103397320,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,535 000 ₽
2286,лифтбек,серый,"4 собственника,птс оригинал( таможенный), в дт...",1.6 л / 102 л.с. / Бензин,1.6 л,102 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",211 000 км,Б/у,...,№ 1103008511,механическая,3 или более,7 лет,Оригинал,передний,Левый,Не требует ремонта,Растаможен,270 000 ₽


In [48]:
df_audi = pd.read_csv('audi.csv')
df_audi

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price,Владение
0,купе,чёрный,Состояние нового автомобиля. Машина суппер! Оч...,2.0 л / 249 л.с. / Бензин,2.0 л,249 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",2 500 км,Б/у,...,№ 1103153653,роботизированная,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,3 250 000 ₽,NaN
1,внедорожник 5 дв.,чёрный,Автомобиль в отличном состоянии. Гаражное хран...,3.0 л / 249 л.с. / Дизель,3.0 л,249 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",179 000 км,Б/у,...,№ 1102867612,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 980 000 ₽,4 года и 5 месяцев
2,седан,белый,NaN,2.0 л / 180 л.с. / Бензин,2.0 л,180 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",105 000 км,Б/у,...,№ 1103196050,вариатор,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,880 000 ₽,3 месяца
3,кабриолет,красный,Полный Эксклюзив Единственная в РоссииМашина П...,5.2 л / 525 л.с. / Бензин,5.2 л,525 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",36 000 км,Б/у,...,№ 1102811559,роботизированная,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,6 000 000 ₽,NaN
4,внедорожник 5 дв.,белый,"В продаже автомобиль от компании АО ABTODOM, о...",3.0 л / 245 л.с. / Дизель,3.0 л,245 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",48 795 км,Б/у,...,№ 1102869261,автоматическая,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 370 000 ₽,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,седан,синий,"День добрый , продаю машину не спеша За время ...",1.6 л / 101 л.с. / Бензин,1.6 л,101 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",333 333 км,Б/у,...,№ 1103337578,механическая,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,249 999 ₽,NaN
2800,седан,белый,"Торг уместен. Машина в нормальном состоянии, е...",1.8 л / 75 л.с. / Бензин,1.8 л,75 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",242 840 км,Б/у,...,№ 1102979815,механическая,3 или более,Дубликат,передний,Левый,Не требует ремонта,Растаможен,70 000 ₽,NaN
2801,седан,серый,Продам автомобиль в хорошем состоянии. На ходу...,2.0 л / 130 л.с. / Бензин,2.0 л,130 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",380 000 км,Б/у,...,№ 1103263365,механическая,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,270 000 ₽,7 месяцев
2802,седан,красный,Мотор после капремонта. Есть недочёты по кузов...,2.0 л / 101 л.с. / Бензин,2.0 л,101 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",400 000 км,Б/у,...,№ 1103464241,механическая,3 или более,Дубликат,передний,Левый,Не требует ремонта,Растаможен,145 000 ₽,NaN


In [49]:
df_bmv = pd.read_csv('bmv.csv')
df_bmv

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price,Владение
0,внедорожник 5 дв.,серый,"В продаже автомобили от компании АО ABTODOM, о...",3.0 л / 249 л.с. / Дизель,3.0 л,249 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",24 243 км,Б/у,...,№ 1102499386,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,7 375 840 ₽,NaN
1,хэтчбек 5 дв.,белый,"Немецкая сборка! Машина в отличном состоянии, ...",1.5 л / 136 л.с. / Бензин,1.5 л,136 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",100 530 км,Б/у,...,№ 1102638350,автоматическая,3 или более,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1 250 000 ₽,5 лет и 2 месяца
2,седан,красный,Машина практически новаяСостояние - идеальное ...,4.4 л / 625 л.с. / Бензин,4.4 л,625 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",3 000 км,Б/у,...,№ 1103407832,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,11 600 000 ₽,5 месяцев
3,внедорожник 5 дв.,белый,BMW G07 SKD X7 xDrive30d M Sport ProПАКЕТ BMW ...,3.0 л / 249 л.с. / Дизель,3.0 л,249 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",7 км,Б/у,...,№ 1102869987,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,8 197 000 ₽,2 месяца
4,внедорожник 5 дв.,чёрный,"В продаже автомобили от компании АО ABTODOM, о...",3.0 л / 249 л.с. / Дизель,3.0 л,249 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",43 782 км,Б/у,...,№ 1102222500,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 749 000 ₽,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3599,внедорожник 5 дв.,синий,Автомобиль представлен по программе Premium Se...,2.0 л / 184 л.с. / Бензин,2.0 л,184 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",8 550 км,Б/у,...,№ 1102832386,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,3 530 000 ₽,NaN
3600,внедорожник 5 дв.,синий,«ЛУКАВТО» – официальный дилер Mercedes-Benz в ...,3.0 л / 218 л.с. / Дизель,3.0 л,218 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",55 865 км,Б/у,...,№ 1103365556,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,3 549 000 ₽,NaN
3601,седан,чёрный,"машина в идеальном состоянии по технике,по куз...",1.6 л / 102 л.с. / Бензин,1.6 л,102 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",110 000 км,Б/у,...,№ 1103496289,механическая,3 или более,Оригинал,задний,Левый,Не требует ремонта,Растаможен,185 000 ₽,NaN
3602,внедорожник 5 дв.,чёрный,Если Вы хотите приобрести автомобиль с пробего...,3.0 л / 381 л.с. / Дизель,3.0 л,381 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",92 222 км,Б/у,...,№ 1103454720,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,4 199 000 ₽,NaN


In [50]:
df_honda = pd.read_csv('honda.csv')
df_honda

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price,Владение
0,внедорожник 5 дв.,чёрный,Продается действительно честный Honda CR-V в и...,2.4 л / 190 л.с. / Бензин,2.4 л,190 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",88 000 км,Б/у,...,№ 1103275642,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 700 000 ₽,NaN
1,внедорожник 5 дв.,чёрный,NaN,3.5 л / 281 л.с. / Бензин,3.5 л,281 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",136 000 км,Б/у,...,№ 1103199891,автоматическая,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 400 000 ₽,4 месяца
2,внедорожник 5 дв.,чёрный,Уникальное предложение на автомобили с пробего...,2.4 л / 186 л.с. / Бензин,2.4 л,186 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",116 250 км,Б/у,...,№ 1102811330,вариатор,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 099 000 ₽,NaN
3,внедорожник 5 дв.,синий,Автомобиль проверен и готов к эксплуатацииГара...,2.0 л / 147 л.с. / Бензин,2.0 л,147 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",222 000 км,Б/у,...,№ 1102631654,автоматическая,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,302 337 ₽,NaN
4,внедорожник 5 дв.,чёрный,ПРИ ПОКУПКЕ АВТОМОБИЛЯ В КРЕДИТ ПОЛУЧИТЕ СКИДК...,2.4 л / 166 л.с. / Бензин,2.4 л,166 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",146 800 км,Б/у,...,№ 1102299464,автоматическая,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,999 000 ₽,10 лет и 6 месяцев
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,хэтчбек 5 дв.,серый,Реальный пробег. В моих руках почти 4 года. За...,1.8 л / 140 л.с. / Бензин,1.8 л,140 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",237 000 км,Б/у,...,№ 1103429635,механическая,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,456 000 ₽,3 года и 6 месяцев
966,седан,серый,"Продаю авто 2007 гвНе кредитный, штрафов и огр...",1.8 л / 140 л.с. / Бензин,1.8 л,140 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",130 000 км,Б/у,...,№ 1103296965,автоматическая,3 или более,Дубликат,передний,Левый,Не требует ремонта,Растаможен,465 000 ₽,4 месяца
967,внедорожник 5 дв.,зелёный,NaN,2.0 л / 147 л.с. / Бензин,2.0 л,147 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",350 000 км,Б/у,...,№ 1103409619,механическая,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,315 000 ₽,NaN
968,хэтчбек 5 дв.,зелёный,Машина на ходу. Замечания по кузову. ABC отклю...,1.4 л / 90 л.с. / Бензин,1.4 л,90 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",425 000 км,Б/у,...,№ 1103323060,механическая,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,150 000 ₽,7 лет и 11 месяцев


In [51]:
df_infininti = pd.read_csv('infiniti.csv')
df_infininti

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,внедорожник 5 дв.,белый,Комплектация Sensory. Дистанционный запуск дви...,2.0 л / 272 л.с. / Бензин,2.0 л,272 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",15 900 км,Б/у,...,№ 1103213023,вариатор,1 владелец,1 год и 1 месяц,Оригинал,передний,Левый,Не требует ремонта,Растаможен,2 650 000 ₽
1,внедорожник 5 дв.,чёрный,NaN,5.6 л / 405 л.с. / Бензин,5.6 л,405 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",98 000 км,Б/у,...,№ 1103201900,автоматическая,3 или более,4 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 380 000 ₽
2,внедорожник 5 дв.,чёрный,ГОД ВЫПУСКА 2014. ПТС ВЫДАН В 2015 ГОДУ. ЭКСПЛ...,5.6 л / 405 л.с. / Бензин,5.6 л,405 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",84 300 км,Б/у,...,№ 1103211535,автоматическая,1 владелец,6 лет и 2 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 950 000 ₽
3,седан,белый,Продажа от официального дилера ГК «АВТОМИР».► ...,2.5 л / 222 л.с. / Бензин,2.5 л,222 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",208 704 км,Б/у,...,№ 1102616067,автоматическая,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,710 000 ₽
4,внедорожник 5 дв.,чёрный,Автомобиль 2017 года выпуска .Куплен и начал э...,5.6 л / 405 л.с. / Бензин,5.6 л,405 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",39 000 км,Б/у,...,№ 1102860202,автоматическая,1 владелец,3 года,Оригинал,полный,Левый,Не требует ремонта,Растаможен,3 669 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,седан,серый,Японский премиум на полном ходу.невероятная ди...,3.5 л / 280 л.с. / Бензин,3.5 л,280 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",290 000 км,Б/у,...,№ 1103079219,автоматическая,3 или более,NaN,Дубликат,полный,Левый,Не требует ремонта,Растаможен,500 000 ₽
704,внедорожник 5 дв.,синий,"Продаю собственный авто, я владелец, вписан в ...",3.5 л / 280 л.с. / Бензин,3.5 л,280 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",216 000 км,Б/у,...,№ 1103093111,автоматическая,3 или более,7 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,710 000 ₽
705,седан,чёрный,Авто в идеальном техническом состоянии!!! Кожа...,2.5 л / 222 л.с. / Бензин,2.5 л,222 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",165 000 км,Б/у,...,№ 1086934961,автоматическая,3 или более,NaN,Дубликат,задний,Левый,Не требует ремонта,Растаможен,1 000 000 ₽
706,внедорожник 5 дв.,чёрный,Oтличный паркeтник для гoрода приспосoблен нa ...,3.5 л / 280 л.с. / Бензин,3.5 л,280 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",200 000 км,Б/у,...,№ 1103175562,автоматическая,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,760 000 ₽


In [52]:
df_lexus = pd.read_csv('lexus.csv')
df_lexus

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,внедорожник 5 дв.,белый,"Машина в отличном состоянии, почти новый автом...",2.0 л / 150 л.с. / Бензин,2.0 л,150 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",11 000 км,Б/у,...,№ 1103333114,вариатор,3 или более,3 года и 6 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 850 000 ₽
1,внедорожник 5 дв.,чёрный,ВЫГОДА ПРИ ОБМЕНЕ ДО 50000 РУБЛЕЙ!*ВЫГОДА ПРИ ...,4.5 л / 272 л.с. / Дизель,4.5 л,272 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",71 000 км,Б/у,...,№ 1102802932,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,5 450 000 ₽
2,седан,чёрный,NaN,2.5 л / 184 л.с. / Бензин,2.5 л,184 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",168 000 км,Б/у,...,№ 1103202044,автоматическая,2 владельца,4 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 520 000 ₽
3,внедорожник 5 дв.,серебристый,Единственный и неповторимый. Куплен в 2015 год...,2.0 л / 238 л.с. / Бензин,2.0 л,238 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",117 000 км,Б/у,...,№ 1102465400,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 300 000 ₽
4,внедорожник 5 дв.,белый,Ваш выбор и Ваше время — наши главные приорите...,2.0 л / 150 л.с. / Бензин,2.0 л,150 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",28 350 км,Б/у,...,№ 1102875194,вариатор,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 750 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,внедорожник 5 дв.,чёрный,"ПТС оригинал, ключи, брелоки в полном комплект...",4.6 л / 296 л.с. / Бензин,4.6 л,296 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",208 000 км,Б/у,...,№ 1103193811,автоматическая,3 или более,4 года и 6 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 190 000 ₽
936,внедорожник 5 дв.,чёрный,Машина огонь! За время пользования только поло...,3.5 л / 277 л.с. / Бензин,3.5 л,277 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",295 255 км,Б/у,...,№ 1102917599,автоматическая,3 или более,4 года,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 800 000 ₽
937,внедорожник 5 дв.,голубой,Владею авто 6 лет. Что сделано: заменён радиат...,3.3 л / 230 л.с. / Бензин,3.3 л,230 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",204 150 км,Б/у,...,№ 1103181466,автоматическая,3 или более,NaN,Дубликат,полный,Левый,Не требует ремонта,Растаможен,780 000 ₽
938,седан,голубой,Был куплен мной 10 апреля 2019 года с пробегом...,2.5 л / 208 л.с. / Бензин,2.5 л,208 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",293 000 км,Б/у,...,№ 1103170216,автоматическая,3 или более,2 года и 1 месяц,Дубликат,задний,Левый,Не требует ремонта,Растаможен,940 000 ₽


In [53]:
df_mersedes = pd.read_csv('mersedes.csv')
df_mersedes

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,седан,белый,"Машина в прекрасном состоянии, не требует ника...",1.6 л / 156 л.с. / Бензин,1.6 л,156 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",92 000 км,Б/у,...,№ 1102406138,роботизированная,2 владельца,2 года и 10 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 450 000 ₽
1,минивэн,белый,"MERCEDES-BENZ VITO TOURER 116 CDI 4x4 , 163 л....",2.1 л / 163 л.с. / Дизель,2.1 л,163 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",24 910 км,Б/у,...,№ 1103033765,автоматическая,3 или более,2 года и 1 месяц,Оригинал,полный,Левый,Не требует ремонта,Растаможен,3 100 000 ₽
2,купе,чёрный,Состояние нового автомобиля!Комплектация AMG G...,4.0 л / 585 л.с. / Бензин,4.0 л,585 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",8 000 км,Б/у,...,№ 1102824245,роботизированная,2 владельца,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,12 500 000 ₽
3,купе,жёлтый,Авто в отличном состоянии. Вложений не требует...,4.0 л / 510 л.с. / Бензин,4.0 л,510 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",12 000 км,Б/у,...,№ 1102842142,роботизированная,2 владельца,3 года и 3 месяца,Оригинал,задний,Левый,Не требует ремонта,Растаможен,8 000 000 ₽
4,седан,серый,Помимо того что машина в самой максимальной ко...,4.0 л / 612 л.с. / Бензин,4.0 л,612 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",4 633 км,Б/у,...,№ 1102825182,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,12 400 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3594,седан,серый,Отличный и ухоженный автомобиль. Я второй собс...,2.0 л / 184 л.с. / Бензин,2.0 л,184 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",62 600 км,Б/у,...,№ 1103297996,автоматическая,2 владельца,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1 790 000 ₽
3595,седан,белый,Покупал Авто в январе 2021 года. Автомобиль в ...,1.6 л / 156 л.с. / Бензин,1.6 л,156 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",78 458 км,Б/у,...,№ 1103044626,автоматическая,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1 970 000 ₽
3596,внедорожник 5 дв.,чёрный,Автомобиль в отличном состоянии! Перекупам и с...,3.5 л / 306 л.с. / Бензин,3.5 л,306 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",119 000 км,Б/у,...,№ 1090703716,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 330 000 ₽
3597,седан,белый,"Отличный автомобиль. Два владельца, но по факт...",1.8 л / 184 л.с. / Бензин,1.8 л,184 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",83 000 км,Б/у,...,№ 1103218638,автоматическая,2 владельца,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1 400 000 ₽


In [54]:
df_mitsubishi = pd.read_csv('mitsubishi.csv')
df_mitsubishi

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,внедорожник 5 дв.,чёрный,"Авто в отличном состоянии, вложений не требует...",2.0 л / 146 л.с. / Бензин,2.0 л,146 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",236 000 км,Б/у,...,№ 1102449902,вариатор,1 владелец,8 лет и 7 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,928 000 ₽
1,внедорожник 5 дв.,коричневый,"Автомобиль в отличном состоянии, технически ис...",3.0 л / 178 л.с. / Бензин,3.0 л,178 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",211 060 км,Б/у,...,№ 1103035505,автоматическая,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 695 000 ₽
2,седан,серебристый,⚡⚡⚡ ВЫ МОЖЕТЕ ПОЛУЧИТЬ СКИДКУ НА ДАННЫЙ АВТОМО...,1.6 л / 98 л.с. / Бензин,1.6 л,98 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",188 000 км,Б/у,...,№ 1103380083,автоматическая,3 или более,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,195 000 ₽
3,внедорожник 5 дв.,чёрный,Фактически 4 владельца. Максимальная комплекта...,3.0 л / 178 л.с. / Бензин,3.0 л,178 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",156 000 км,Б/у,...,№ 1102748232,автоматическая,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,895 000 ₽
4,внедорожник 5 дв.,чёрный,ЛОТ: 01250566Реутов HYUNDAIВы можете получить ...,2.0 л / 146 л.с. / Бензин,2.0 л,146 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",79 237 км,Б/у,...,№ 1102911289,вариатор,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 500 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105,седан,серебристый,NaN,1.5 л / 109 л.с. / Бензин,1.5 л,109 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",130 000 км,Б/у,...,№ 1103443761,автоматическая,3 или более,2 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,470 000 ₽
2106,седан,чёрный,"Продаётся машина, в нормальном состоянии. Купи...",1.6 л / 98 л.с. / Бензин,1.6 л,98 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",290 000 км,Б/у,...,№ 1103057320,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,260 000 ₽
2107,седан,серебристый,"Продам лансер 10,чистокровный японец, автомат ...",1.5 л / 109 л.с. / Бензин,1.5 л,109 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",120 112 км,Б/у,...,№ 1103440566,автоматическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,430 000 ₽
2108,седан,белый,"Замена гофры + прокладка резонатора 2000, ремо...",1.3 л / 75 л.с. / Бензин,1.3 л,75 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",222 000 км,Б/у,...,№ 1103054430,механическая,3 или более,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,100 000 ₽


In [55]:
df_nissan = pd.read_csv('nissan.csv')
df_nissan

,bodyType,color,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,brand,...,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,description
0,внедорожник 5 дв.,серебристый,2.0 л / 143 л.с. / Бензин,2.0 л,143 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",26 800 км,Б/у,Nissan,...,автоматическая,1 владелец,4 года и 5 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,960 000 ₽,NaN
1,минивэн,чёрный,3.5 л / 240 л.с. / Бензин,3.5 л,240 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",137 000 км,Б/у,Nissan,...,автоматическая,1 владелец,NaN,Оригинал,задний,Правый,Не требует ремонта,Растаможен,780 000 ₽,"Aвтoмoбиль тoлько пpигнан в PФ, получен таможe..."
2,внедорожник 5 дв.,белый,3.5 л / 265 л.с. / Бензин,3.5 л,265 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",103 000 км,Б/у,Nissan,...,вариатор,3 или более,3 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,690 000 ₽,NaN
3,купе,оранжевый,3.8 л / 555 л.с. / Бензин,3.8 л,555 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",40 000 км,Б/у,Nissan,...,роботизированная,2 владельца,2 года и 1 месяц,Оригинал,полный,Левый,Не требует ремонта,Растаможен,6 450 000 ₽,Автомобиль в отличном состоянии: без техническ...
4,внедорожник 5 дв.,белый,2.0 л / 144 л.с. / Бензин,2.0 л,144 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",14 500 км,Б/у,Nissan,...,вариатор,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 184 000 ₽,Автомобиль после ДТП.( Восстановлена некачеств...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3460,седан,серебристый,1.5 л / 98 л.с. / Бензин,1.5 л,98 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",230 000 км,Б/у,Nissan,...,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,190 000 ₽,"Надежный, экономичный, неприхотливый японский ..."
3461,внедорожник 5 дв.,чёрный,2.0 л / 140 л.с. / Бензин,2.0 л,140 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",284 000 км,Б/у,Nissan,...,механическая,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,500 000 ₽,Машина в очень хорошем для своих лет состояние...
3462,седан,красный,1.6 л / 109 л.с. / Бензин,1.6 л,109 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",340 000 км,Б/у,Nissan,...,механическая,3 или более,4 месяца,Дубликат,передний,Левый,Не требует ремонта,Растаможен,290 000 ₽,Для своих лет в хорошем состоянии. Есть не бол...
3463,седан,пурпурный,2.0 л / 140 л.с. / Бензин,2.0 л,140 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",300 000 км,Б/у,Nissan,...,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,200 000 ₽,"Продаю нисан Максима а32, проделано много рабо..."


In [56]:
df_toyota =pd.read_csv('toyota.csv')
df_toyota

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,седан,коричневый,Один собственник! (Одна запись в оригинальном ...,2.5 л / 181 л.с. / Бензин,2.5 л,181 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",75 000 км,Б/у,...,№ 1103323286,автоматическая,1 владелец,3 года и 7 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 750 000 ₽
1,седан,белый,"Добрый день!ПТС оригинал , полный комплект док...",2.5 л / 181 л.с. / Бензин,2.5 л,181 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",136 000 км,Б/у,...,№ 1103158568,автоматическая,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 548 000 ₽
2,внедорожник 5 дв.,белый,Учетный № 7933Комплектация «Люкс»:• Антиблокир...,3.5 л / 273 л.с. / Бензин,3.5 л,273 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",118 613 км,Б/у,...,№ 1103450306,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 710 000 ₽
3,седан,чёрный,Автомобиль в хорошем состоянии. Полный пакет д...,2.4 л / 167 л.с. / Бензин,2.4 л,167 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",165 200 км,Б/у,...,№ 1102793844,автоматическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,765 000 ₽
4,седан,чёрный,- выгода 60 000 рублей при покупке авто в Trad...,2.0 л / 147 л.с. / Бензин,2.0 л,147 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",137 000 км,Б/у,...,№ 1102894747,автоматическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,549 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591,компактвэн,синий,"День добрый!Продам ""возможно"" ББиху 2002г Тех...",1.5 л / 105 л.с. / Бензин,1.5 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",170 000 км,Б/у,...,№ 1103109635,автоматическая,3 или более,9 месяцев,Дубликат,передний,Правый,Не требует ремонта,Растаможен,250 000 ₽
3592,внедорожник 5 дв.,чёрный,"Машина в хорошем состоянии, использовалась тол...",2.5 л / 180 л.с. / Бензин,2.5 л,180 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",79 000 км,Б/у,...,№ 1103252400,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 950 000 ₽
3593,седан,белый,Автомобиль в максимальной комплектации. Камера...,1.8 л / 140 л.с. / Бензин,1.8 л,140 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",220 000 км,Б/у,...,№ 1103379791,вариатор,1 владелец,4 года и 1 месяц,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 150 000 ₽
3594,седан,синий,Автомобиль в максимальной комплектации. Состоя...,2.5 л / 181 л.с. / Бензин,2.5 л,181 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",33 000 км,Б/у,...,№ 1103335070,автоматическая,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,2 290 000 ₽


In [57]:
df_volvo = pd.read_csv('volvo.csv')
df_volvo

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,хэтчбек 5 дв.,белый,Один хозяин. Оригинал ПТС. Состояние нового ав...,1.5 л / 152 л.с. / Бензин,1.5 л,152 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",45 000 км,Б/у,...,№ 1103342890,автоматическая,1 владелец,3 года и 7 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 640 000 ₽
1,хэтчбек 5 дв.,чёрный,"Стильный, безопасный, динамичный и надежный ав...",2.0 л / 180 л.с. / Бензин,2.0 л,180 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",135 000 км,Б/у,...,№ 1103463370,автоматическая,3 или более,1 год,Оригинал,передний,Левый,Не требует ремонта,Растаможен,885 000 ₽
2,внедорожник 5 дв.,синий,Вольво хс90 V8 R-design. Цвет глубокий черно-с...,4.4 л / 315 л.с. / Бензин,4.4 л,315 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",268 512 км,Б/у,...,№ 1102895637,автоматическая,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 250 000 ₽
3,внедорожник 5 дв.,чёрный,Автомобиль приобретался у официального дилера....,2.4 л / 200 л.с. / Дизель,2.4 л,200 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",116 000 км,Б/у,...,№ 1102909423,автоматическая,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 570 000 ₽
4,внедорожник 5 дв.,белый,"Все автомобили, представленные в продаже, прох...",2.4 л / 163 л.с. / Дизель,2.4 л,163 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",106 000 км,Б/у,...,№ 1101934378,автоматическая,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 095 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,седан,белый,"Автомобиль на ходу,требуется ремонт по кузову....",2.3 л / 131 л.с. / Бензин,2.3 л,131 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",332 000 км,Б/у,...,№ 1103141458,механическая,3 или более,NaN,Дубликат,задний,Левый,Не требует ремонта,Растаможен,100 000 ₽
1121,седан,голубой,"НЕ ТОРГУЮСЬ. Пока вы это читаете, автомобиль ...",2.4 л / 140 л.с. / Бензин,2.4 л,140 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",211 695 км,Б/у,...,№ 1103290347,механическая,3 или более,2 года и 7 месяцев,Дубликат,передний,Левый,Не требует ремонта,Растаможен,405 000 ₽
1122,седан,серый,"Авто в хорошем состоянии, ремонта не требует. ...",2.4 л / 170 л.с. / Бензин,2.4 л,170 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",174 000 км,Б/у,...,№ 1103135856,автоматическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,360 000 ₽
1123,седан,серый,СРОЧНО!!! Автомобиль хороший служил верой и пр...,2.4 л / 170 л.с. / Бензин,2.4 л,170 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",270 000 км,Б/у,...,№ 1103301533,автоматическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,300 000 ₽


In [58]:
df_wolkswagen = pd.read_csv('wolkswagen.csv')
df_wolkswagen 

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,внедорожник 5 дв.,чёрный,Автомобиль в отличном состояний вложений не тр...,2.5 л / 174 л.с. / Дизель,2.5 л,174 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",183 438 км,Б/у,...,№ 1102836072,автоматическая,3 или более,17 лет и 1 месяц,Оригинал,полный,Левый,Не требует ремонта,Растаможен,550 555 ₽
1,внедорожник 5 дв.,коричневый,Авто полностью в родной краске(смотрите прило...,3.0 л / 204 л.с. / Дизель,3.0 л,204 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",175 000 км,Б/у,...,№ 1102281005,автоматическая,3 или более,8 лет и 5 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 499 999 ₽
2,внедорожник 5 дв.,белый,✔ Один владелец✔ ПТС оригинал✔ Отличное состоя...,1.4 л / 150 л.с. / Бензин,1.4 л,150 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",31 000 км,Б/у,...,№ 1102907371,роботизированная,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1 735 000 ₽
3,внедорожник 5 дв.,чёрный,"В продаже автомобили от компании АО ABTODOM, о...",3.0 л / 204 л.с. / Дизель,3.0 л,204 л.с.,Дизель,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",89 107 км,Б/у,...,№ 1102539652,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 755 000 ₽
4,внедорожник 5 дв.,белый,ЛОТ: 01250946FAVORIT MOTORS MAZDA МКАДВы может...,2.0 л / 170 л.с. / Бензин,2.0 л,170 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",78 660 км,Б/у,...,№ 1102910719,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,890 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3388,седан,белый,NaN,1.6 л / 105 л.с. / Бензин,1.6 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",182 000 км,Б/у,...,№ 1102633901,автоматическая,1 владелец,6 лет и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,570 000 ₽
3389,седан,белый,Продаётся личный автомобиль. Использовался в с...,1.6 л / 110 л.с. / Бензин,1.6 л,110 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",107 112 км,Б/у,...,№ 1101614463,автоматическая,1 владелец,1 год и 8 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,845 000 ₽
3390,седан,серебристый,«Сток Авто» - автомобили с пробегом и репутаци...,1.6 л / 105 л.с. / Бензин,1.6 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",90 000 км,Б/у,...,№ 1103186043,механическая,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,605 000 ₽
3391,седан,чёрный,.✔ Автомобиль прошёл юридическую проверку и по...,1.6 л / 102 л.с. / Бензин,1.6 л,102 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",119 000 км,Б/у,...,№ 1103187466,роботизированная,3 или более,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,455 000 ₽


In [39]:
train = pd.concat([df_scoda, df_audi, df_bmv, df_honda, df_infininti, df_lexus, df_mersedes, df_mitsubishi, df_nissan,
                  df_toyota, df_volvo, df_wolkswagen])

In [40]:
train

,bodyType,color,description,engine,engineDisplacement,enginePower,fuelType,equipment_dict,mileage,card_type,...,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,лифтбек,жёлтый,Остался один автомобиль!!! Автомобили были зам...,1.6 л / 110 л.с. / Бензин,1.6 л,110 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",125 000 км,Б/у,...,№ 1103050112,автоматическая,1 владелец,2 года и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,930 000 ₽
1,лифтбек,серый,"Комплектация ""HOCKEY EDITION"". Куплен в конце ...",1.4 л / 150 л.с. / Бензин,1.4 л,150 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",22 143 км,Б/у,...,№ 1102189226,роботизированная,1 владелец,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,1 545 000 ₽
2,компактвэн,серый,"Хороший рабочий автомобиль, удобный для семьи....",1.6 л / 105 л.с. / Бензин,1.6 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",158 000 км,Б/у,...,№ 1103351952,механическая,2 владельца,8 лет и 3 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,490 000 ₽
3,универсал 5 дв.,серебристый,Автомобиль в отличном состоянии! Обслуживался ...,1.8 л / 150 л.с. / Бензин,1.8 л,150 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",161 000 км,Б/у,...,№ 1102867943,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,265 000 ₽
4,лифтбек,чёрный,Наш автосалон обеспечивает максимальную безопа...,1.6 л / 115 л.с. / Бензин,1.6 л,115 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",152 654 км,Б/у,...,№ 1102818901,автоматическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,335 000 ₽
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3388,седан,белый,NaN,1.6 л / 105 л.с. / Бензин,1.6 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",182 000 км,Б/у,...,№ 1102633901,автоматическая,1 владелец,6 лет и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,570 000 ₽
3389,седан,белый,Продаётся личный автомобиль. Использовался в с...,1.6 л / 110 л.с. / Бензин,1.6 л,110 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",107 112 км,Б/у,...,№ 1101614463,автоматическая,1 владелец,1 год и 8 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,845 000 ₽
3390,седан,серебристый,«Сток Авто» - автомобили с пробегом и репутаци...,1.6 л / 105 л.с. / Бензин,1.6 л,105 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",90 000 км,Б/у,...,№ 1103186043,механическая,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,605 000 ₽
3391,седан,чёрный,.✔ Автомобиль прошёл юридическую проверку и по...,1.6 л / 102 л.с. / Бензин,1.6 л,102 л.с.,Бензин,"{'asciiCat': 'cars', 'category': 'cars', 'engi...",119 000 км,Б/у,...,№ 1103187466,роботизированная,3 или более,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,455 000 ₽


In [41]:
train.brand.unique()

array(['Skoda', 'Audi', 'BMW', 'Honda', 'Infiniti', 'Lexus',
       'Mercedes-Benz', 'Mitsubishi', 'Nissan', 'Toyota', 'Volvo',
       'Volkswagen'], dtype=object)

In [42]:
train = train.to_csv('train.csv', index = False)

In [43]:
train = pd.read_csv('train.csv')

In [59]:
train.brand.unique().count()

AttributeError: 'numpy.ndarray' object has no attribute 'count'